In [ ]:
from YOLO_videoMAE import extract_frames, get_bbox, evaluate, VideoDataset


In [ ]:
video_file = "C:/Users/user/Downloads/2_1.mp4"  
output_dir = "output_frames"    

extract_frames(video_file, output_dir)

In [ ]:
import yaml

with open("./RSL.yaml") as f:
    config = yaml.safe_load(f)

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

In [ ]:
detections = get_bbox(output_dir, model)

In [ ]:
from torchvision import transforms
import torch

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.clamp(x * 1.3, 0, 1)),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])


test_dataset = VideoDataset(
    frames_dir="C:/Users/user/Downloads/src/output_frames" ,
    anno=detections,
    clip_len=8,
    transform=transform
)

In [ ]:
from torch.utils.data import DataLoader

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [ ]:
from transformers import VideoMAEForVideoClassification
import torch.nn as nn

model = VideoMAEForVideoClassification.from_pretrained(
    config['vit']['pr_model'],
    num_labels=33,
    ignore_mismatched_sizes=True,
    num_frames=8,
    attention_probs_dropout_prob=0.1,
    hidden_dropout_prob=0.2
)

model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, model.config.num_labels )
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
evaluate(model, test_loader, device)